## LangChain Tool calling

- Tool calling 은 Chat model 이 "도구 호출" 을 통해 주어진 프롬프트에 응답을 가능하게 함
- 이름만 보면 무언가 실행할 것 같지만, 그렇지 않고 tool 에 인수(arguments)를 생성해서 전달
- 일반적 기술로 model로 부터 구조화된 output을 생성하고 의도하지 않았더라도 이것을 사용할 수 있음

실사용 사례로 비정형 텍스트로 부터 구조화된 output을 추출하는 것 

![Tool Calling](https://python.langchain.com/v0.2/assets/images/tool_call-8d4a8b18e90cacd03f62e94071eceace.png)


### tool schema 정의

- model이 tool calling 을 가능하게 하기 위해 tool 이 수행하는 기능을 기술과 인수들을 정의한 tool schemas 를 전달해야 함
- ChatModel의 경우 bind_tools() 메소드를 이용해 model 에 tool schema 전달
- Tool Schema 는 파이썬 함수, Pydantic Models TypedDict class 또는 LangChain Tool objects 로 전달할 수 있으며, 이후 Model을 호출하면 프롬프트와 함께 이러한 Tool schema 가 전달

#### Python 함수로 정의한 tool schema

In [ ]:
# The function name, type hints, and docstring are all part of the tool schema that's passed to the model. 
# Defining good, descriptive schemas is an extension of prompt engineering and is an important part of
# getting models to perform well.
# docstring 정의 중요 : 기능, 인수 설명
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

#### LangChain Tool

@tool 데코레이터 이용

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


#### Pydantic class

Pydantic 을 이용해 함께 제공하는(accompanying) 함수 없이도 schema 를 동등하게 정의할 수 있음

- 열들은 기본 값을 정의하지 않으면 모든 필수입력(required)

In [1]:
# BaseModel : Pydantic으로 정의하는 기본 모델, Field : 열 정의할 때 사용 (열 자료형 정의와 description 정의)
from langchain_core.pydantic_v1 import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

/Users/yoonani/Development/follow/deep_research_from_scratch/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


#### TypedDict class

> langchain-core>=0.2.25 
> TypedDics 와 annotaions 를 사용할 수 있음


In [9]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(BaseModel):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

### LLM with bind_tools()

In [3]:
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=os.environ["GOOGLE_API_KEY"])

In [10]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--8cf47b69-6ed0-4de0-aeed-b85ac79d4a46-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '98455d87-f586-4328-8e59-432e5a2762ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 102, 'output_tokens': 96, 'total_tokens': 198, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 77}})

#### Tool calls

In [11]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': 'a89c8c91-ff12-4fc8-b0e5-2454f9ac1101',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': 'af1e4aa3-f2db-4312-9fde-52d8407e56d6',
  'type': 'tool_call'}]

#### parsing

- 원하는 경우 출력 파서를 통해 출력을 추가로 처리할 수 있음
- 예를 들어, .tool_calls에 채워진 기존 값을 PydanticToolsParser를 사용하여 Pydantic 객체로 변환할 수 있음

In [13]:
from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools = llm.bind_tools( [add, multiply] )

chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12)]

## tool output 을 chat model에 전달하는 방법

![](https://python.langchain.com/v0.2/assets/images/tool_invocation-7f277888701ee431a17607f1a035c080.png)

![](https://python.langchain.com/v0.2/assets/images/tool_results-71b4b90f33a56563c102d91e7821a993.png)

In [18]:
from langchain_core.tools import tool

# llm 정의
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key = os.environ["GOOGLE_API_KEY"] )


# tool 정의 : decorator 사용
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


# tool list
tools = [add, multiply]

# llm 에 tool 바인딩
llm_with_tools = llm.bind_tools(tools)



In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

# tool 을 바인딩한 model 실행 : HumanMessage 로 전달
ai_msg = llm_with_tools.invoke(messages)

#################################
# tool_calls : tool calling 실행 #
#################################
print(ai_msg.tool_calls)

# tool calling 으로 생성한 정보를 message 에 추가
messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '071f596e-8ab7-4b6c-8687-4d9e4908cf0b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': 'f9d94ad1-4946-4435-b399-8c5bf3668b09', 'type': 'tool_call'}]


In [ ]:
# tool 의 응답을 message 로 추가하기
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ffab870d-f202-4880-8a3f-01c5268d69ee-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '071f596e-8ab7-4b6c-8687-4d9e4908cf0b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': 'f9d94ad1-4946-4435-b399-8c5bf3668b09', 'type': 'tool_call'}], usage_metadata={'input_tokens': 116, 'output_tokens': 110, 'total_tokens': 226, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 71}}),
 ToolMessage(content='36', name='multiply', tool_call_id='071f596e-8ab7-4b6c-8687-4d9e4908cf0b'),
 ToolMessage(content='60', name='add', tool_call

In [ ]:
# tool 을 이용한 응답 생성
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 = 36 and 11 + 49 = 60.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--c5fd2fc5-219a-4999-ae14-3c10e03dbf48-0', usage_metadata={'input_tokens': 181, 'output_tokens': 22, 'total_tokens': 203, 'input_token_details': {'cache_read': 0}})

Note that each ToolMessage must include a tool_call_id that matches an id in the original tool calls that the model generates. This helps the model match tool responses with tool calls.

